In [2]:
%run ../../__init__.py
%load_ext autoreload
%autoreload 2

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Models sizes

In [13]:
def calculate_model_size(checkpoint_name, model):
    param_size = 0
    
    for param in model.values():
        param_size += param.nelement() * param.element_size()
        
    param_size = param_size / 1024**2
    
    print(checkpoint_name, f"{param_size:.2f} MB")

In [14]:
models_path = '../../../runs/models'
checkpoints = ['superpoint/checkpoints/model_superpoint.pt',
               'r2d2/checkpoints/model_r2d2_WAF_N16.pt',
               'keynet/checkpoints/model_keynet_pytorch.pt',
               'disk/checkpoints/model_depth-save.pt',
               'rekd/checkpoints/model_best.pt',
               'shiness/checkpoints/model_r_mAA=0.7706.pt']

for ch in checkpoints:    
    calculate_model_size(ch, torch.load(os.path.join(models_path, ch)))

superpoint/checkpoints/model_superpoint.pt 4.96 MB
r2d2/checkpoints/model_r2d2_WAF_N16.pt 1.85 MB
keynet/checkpoints/model_keynet_pytorch.pt 0.02 MB
disk/checkpoints/model_depth-save.pt 4.17 MB
rekd/checkpoints/model_best.pt 99.12 MB
shiness/checkpoints/model_r_mAA=0.7706.pt 3.54 MB


## Models inference times

In [5]:
from omegaconf import OmegaConf
from hydra import compose, initialize

import source.core.namespace as ns


dataset = 'imc_pt'
models_list = ['superpoint_disk/default',
               'r2d2_disk/default',
               'keynet_disk/default', 
               'disk/default',
               'rekd_disk/default',
               'shiness_disk/default',
               'shiss_disk/default']

inference_times = []
process_times = []

for model in models_list:
    val_overrides = [f"+models/{model}=custom",
                     f"+datasets/{dataset}/setup/ICCV2023/test=time",
                     "+experiment=measure_time"]
    
    with initialize(config_path="../../../config", version_base="1.1"):
        config = compose(overrides=val_overrides)
        
        print(OmegaConf.to_yaml(config))
        
        OmegaConf.set_struct(config, False)
        
        experiment = SummertimeExperiment(config)
        batch, endpoint, metrics = experiment.run()
        
        model_inference_times = []
        
        for k, v in metrics.items():
            if ns.INFERENCE_TIME in k:
                model_inference_times.append((k, v))
                
        model_process_times = []
        
        for k, v in metrics.items():
            if ns.PROCESS_TIME in k:
                model_process_times.append((k, v))
                
        inference_times.append(model_inference_times)
        process_times.append(model_process_times)
        

models:
  superpoint:
    input_channels: 1
    input_size_divisor: 8
    modules:
      backbone:
        idx: 0
        default_idx: 0
        process_idx: 0
      detector:
        idx: 1
        default_idx: 1
        process_idx: 1
    checkpoint:
      rel_path: models/superpoint
      name: superpoint
      remap: true
    eval_params:
      nms_size: 5
      score_thresh: 0.015
  disk:
    input_channels: 3
    input_size_divisor: 16
    modules:
      descriptor:
        idx: 2
        default_idx: 1
        process_idx: 2
    checkpoint:
      name: depth-save
      rel_path: models/disk
      remap: true
    eval_params:
      matcher:
        name: l2
  model_name: superpoint_disk
datasets:
  imc_pt:
    test:
      csv_path: /mnt/sda/datasets/IMCPT/test_pairs.csv
      data_options:
      - image
      process: true
      transforms:
        grayscale: null
        resize:
          width: 640
          height: 480
        crop:
          type: divisor_crop
    evaluation:

/home/konstantin/personal/Summertime/source/utils/endpoint_utils.py:375: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  y = flat_ids // w
/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:4004: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  "Default grid_sample and affine_grid behavior has changed "


models:
  r2d2:
    input_channels: 3
    input_size_divisor: 2
    modules:
      detector:
        idx: 0
        default_idx: 0
        process_idx: 0
    checkpoint:
      rel_path: models/r2d2
      name: r2d2_WAF_N16
      remap: true
    eval_params:
      nms_size: 3
      score_thresh: 0.7
      conf_thresh: 0.7
  disk:
    input_channels: 3
    input_size_divisor: 16
    modules:
      descriptor:
        idx: 1
        default_idx: 1
        process_idx: 1
    checkpoint:
      name: depth-save
      rel_path: models/disk
      remap: true
    eval_params:
      matcher:
        name: l2
  model_name: r2d2_disk
datasets:
  imc_pt:
    test:
      csv_path: /mnt/sda/datasets/IMCPT/test_pairs.csv
      data_options:
      - image
      process: true
      transforms:
        grayscale: null
        resize:
          width: 640
          height: 480
        crop:
          type: divisor_crop
    evaluation:
      test:
        mode_epoch: 1
        metric_log_iter: -1
        m

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


models:
  disk:
    input_channels: 3
    input_size_divisor: 16
    modules:
      detector_descriptor:
        idx: 0
        default_idx: 0
        process_idx: 0
    checkpoint:
      name: depth-save
      rel_path: models/disk
      remap: true
    eval_params:
      nms_size: 5
      matcher:
        name: l2
  model_name: disk
datasets:
  imc_pt:
    test:
      csv_path: /mnt/sda/datasets/IMCPT/test_pairs.csv
      data_options:
      - image
      process: true
      transforms:
        grayscale: null
        resize:
          width: 640
          height: 480
        crop:
          type: divisor_crop
    evaluation:
      test:
        mode_epoch: 1
        metric_log_iter: -1
        metrics:
          time: null
        eval_params:
          topk: 2048
    loader:
      test:
        batch_size: 1
        num_samples: 30
        sampler: start_seq
        start_from: 0
        num_workers: 0
experiment:
  num_epochs: 1
  modes:
  - test-eval
  device: 0
  measure_time: t

/opt/conda/lib/python3.7/site-packages/e2cnn/nn/modules/r2_conv/basisexpansion_singleblock.py:80: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272168290/work/aten/src/ATen/native/IndexingUtils.h:30.)
  full_mask[mask] = norms.to(torch.uint8)


Model checkpoint best is loaded
Model checkpoint depth-save is loaded


Model name:  rekd_disk


Eval params of mode 'test':
nms_size: 15
matcher:
  name: l2
topk: 2048

models:
  shiness:
    input_size_divisor: 16
    modules:
      detector:
        idx: 0
        default_idx: 0
        process_idx: 0
        ness:
          feature_channels: 3
          down:
          - 16
          - 32
          - 64
          - 64
          - 64
          up:
          - 64
          - 64
          - 64
          - 1
          min_ness: 0.0
          max_ness: 32.0
        shi:
          sobel_size: 3
          window_size: 3
          window_cov: 2
          loc: true
    eval_params:
      nms_size: 5
      score_thresh: 0.0
    checkpoint:
      rel_path: models/shiness
      name: r_mAA=0.7706
  disk:
    input_channels: 3
    input_size_divisor: 16
    modules:
      descriptor:
        idx: 1
        default_idx: 1
        process_idx: 1
    checkpoint:
      name: depth-save
      rel_pa

/opt/conda/lib/python3.7/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272168290/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


models:
  shiss:
    input_channels: 1
    modules:
      detector:
        idx: 0
        default_idx: 0
        process_idx: 0
        shi:
          sobel_size: 3
          window_size: 3
          window_cov: 2
          loc: true
        ss:
          scale_factor: 2.0
          num_samples: 100
          block_size: 1024
    eval_params:
      nms_size: 5
      score_thresh: 0.003
  disk:
    input_channels: 3
    input_size_divisor: 16
    modules:
      descriptor:
        idx: 1
        default_idx: 1
        process_idx: 1
    checkpoint:
      name: depth-save
      rel_path: models/disk
      remap: true
    eval_params:
      matcher:
        name: l2
  model_name: shiss_disk
datasets:
  imc_pt:
    test:
      csv_path: /mnt/sda/datasets/IMCPT/test_pairs.csv
      data_options:
      - image
      process: true
      transforms:
        grayscale: null
        resize:
          width: 640
          height: 480
        crop:
          type: divisor_crop
    evaluation:
   

In [6]:
for model, inference_time, process_time in zip(models_list, inference_times, process_times): 
    print(model)
    
    for (module, time_list) in inference_time:
        time = sum(time_list[1:]) / len(time_list[1:])        
        print(f"\t {module} {time:.1f} ms")
        
    print('\n')
        
    for (module, time_list) in process_time:
        time = sum(time_list[1:]) / len(time_list[1:])
        print(f"\t {module} {time:.1f} ms")
        
    print('\n')
    print('-----------------------------------------------')
    
    

superpoint_disk/default
	 BackboneWrapper_inference_time 3.7 ms
	 DetectorWrapper_inference_time 0.3 ms
	 DescriptorWrapper_inference_time 19.5 ms


	 BackboneWrapper_process_time 0.0 ms
	 DetectorWrapper_process_time 1.7 ms
	 DescriptorWrapper_process_time 0.4 ms


-----------------------------------------------
r2d2_disk/default
	 DetectorWrapper_inference_time 24.4 ms
	 DescriptorWrapper_inference_time 19.9 ms


	 DetectorWrapper_process_time 2.2 ms
	 DescriptorWrapper_process_time 0.4 ms


-----------------------------------------------
keynet_disk/default
	 DetectorWrapper_inference_time 5.4 ms
	 DescriptorWrapper_inference_time 19.5 ms


	 DetectorWrapper_process_time 2.2 ms
	 DescriptorWrapper_process_time 0.4 ms


-----------------------------------------------
disk/default
	 DetectorDescriptorWrapper_inference_time 19.5 ms


	 DetectorDescriptorWrapper_process_time 0.7 ms


-----------------------------------------------
rekd_disk/default
	 DetectorWrapper_inference_time 59.5 